## Описание проекта<a id="description"></a>
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
Инструкция по выполнению
- [Предобработка данных](#begin)
- [Пропущенные значения](#empty)
##### После [удаления дубликатов](#duplicated) сделайте следующее:
3. [Ответьте на вопросы](#answer)
Ответы на вопросы можно разместить в ячейках тетрадок Jupyter Notebook с типом markdown.
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?
Ответы сопроводите интерпретацией — поясните, о чём именно говорит полученный вами результат.
4. [Напишите общий вывод](#result)
Оформление: Задание выполните в Jupyter Notebook. Программный код заполните в ячейках типа code, текстовые пояснения — в ячейках типа markdown. Примените форматирование и заголовки.


#### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[описание проекта](#description)

## Предообработка данных <a id="begin"></a>

Обработаем данные так чтобы с ними было удобно работать: удалять дубликаты и искать пропущенные значения.     

In [2]:
data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [3]:
data.family_status.unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [4]:
# есть неявные дубликаты за счет заглавных букв
data.family_status = data.family_status.str.lower()
data.family_status.unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [5]:
data.education = data.education.str.lower()
data.education.unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Добавим столбец category_purpose для категоризации значений purpose

In [6]:
data.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В purpose много однокоренных слов, поэтому выделим основные группы: 'ремонт\строительство', 'операции с автомобилем', 'получение образования', 'проведение свадьбы',  'операции с недвижимостью'

In [7]:
def category_purpose(purpose):
    try:
        if ('ремонт' in purpose) | ('строительство' in purpose):
            return 'ремонт\строительство'
        elif 'авто' in purpose:
            return 'операции с автомобилем'
        elif 'образов' in purpose:
            return 'получение образования'
        elif 'свад' in purpose:
            return 'проведение свадьбы'
        elif ('недвиж' in purpose) | ('жиль' in purpose):
            return 'операции с недвижимостью'
        else:
            return 'другое'
    except:
        return 'Error'
    
data['category_purpose'] = data.purpose.apply(category_purpose)
data.groupby('category_purpose').purpose.value_counts()

category_purpose          purpose                               
операции с автомобилем    на покупку своего автомобиля              505
                          автомобиль                                495
                          сделка с подержанным автомобилем          489
                          свой автомобиль                           480
                          на покупку подержанного автомобиля        479
                          автомобили                                478
                          на покупку автомобиля                     472
                          приобретение автомобиля                   462
                          сделка с автомобилем                      455
операции с недвижимостью  операции с недвижимостью                  676
                          покупка коммерческой недвижимости         664
                          операции с жильем                         653
                          покупка жилья для сдачи                   653

In [8]:
data.category_purpose.value_counts()

category_purpose
операции с недвижимостью    8347
операции с автомобилем      4315
получение образования       4022
ремонт\строительство        2493
проведение свадьбы          2348
Name: count, dtype: int64

Проверили что значения purpose соответствуют category_purpose

Проверим минимальный возраст клиента и раскидаем по возрастным категориям остальные для удобства

In [9]:
# Проверяем минимальный возраст клиета и узнаем что и тут есть 0
unknown_age = data[data.dob_years <= 18].debt
# смотрим количество нолей
unknown_age.value_counts()
print(f'Клиентов с возрастом равным 0 всего {round(unknown_age.count()/len(data) * 100, 1)}%')

Клиентов с возрастом равным 0 всего 0.5%


Удалим строки с возрастом меньше 0, так как их достаточно мало и они не смогут существенно повлиять на результат

Возьмем 3 последние категории для возраста из теории психосоциального развития Эрика Эриксона. 
- Молодость (от 19 до 35–39 лет) — начало зрелости, ранние годы семейной жизни, годы до начала среднего возраста
- Взрослость, зрелость (от 35–39 до 60 лет) — период, когда человек прочно связывает себя с определенным родом занятий, а его дети становятся подростками
- Старость (от 60 лет) — период, когда основная работа жизни закончилась

In [10]:
def category_age(years):
    if (years >= 18) & (years <= 35):
        return 'молодой 18+'
    elif (years > 35) & (years < 60):
        return 'взрослый 35+'
    elif years >= 60:
        return 'старый 60+'

data['category_age'] = data.dob_years.apply(category_age)

# Удаляем строки где dob_years = 0 & category_age = None 
data.dropna(subset=['category_age'], inplace=True)
data[data.dob_years == 0].category_age.isna().sum()

0

Преобразуем таблицу и уберем столбцы family_id и education_id так как они дублируют информацию, но в другом формате с которым не удобно работать. 

In [11]:
data.columns

data = data.drop(['family_status_id', 'education_id'], axis=1)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     19260 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   family_status     21424 non-null  object 
 5   gender            21424 non-null  object 
 6   income_type       21424 non-null  object 
 7   debt              21424 non-null  int64  
 8   total_income      19260 non-null  float64
 9   purpose           21424 non-null  object 
 10  category_purpose  21424 non-null  object 
 11  category_age      21424 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 2.1+ MB


## Поиск пропущенных значений<a id="empty"></a>

Теперь обработаем пропуски, тк пропуски не только влияют на анализ данных, но и мешают убирать аномалии
- проверим на NaN и null с помощью isna()
- проверим столбцы с числовыми значениями на 0: days_employed, total_income

In [13]:
data.isna().sum()

children               0
days_employed       2164
dob_years              0
education              0
family_status          0
gender                 0
income_type            0
debt                   0
total_income        2164
purpose                0
category_purpose       0
category_age           0
dtype: int64

[описание проекта](#description)

In [14]:
# смотрю если совпадения в пропущенных значениях
empty_str = data[(data.days_employed.isna()) & (data.total_income.isna())]
print(f'В столбцах days_employed и total_income из {len(data)} найдено {len(empty_str)} совпадений')
empty_str.head()

В столбцах days_employed и total_income из 21424 найдено 2164 совпадений


,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,category_purpose,category_age
12,0,NaN,65,среднее,гражданский брак,M,пенсионер,0,NaN,сыграть свадьбу,проведение свадьбы,старый 60+
26,0,NaN,41,среднее,женат / замужем,M,госслужащий,0,NaN,образование,получение образования,взрослый 35+
29,0,NaN,63,среднее,не женат / не замужем,F,пенсионер,0,NaN,строительство жилой недвижимости,ремонт\строительство,старый 60+
41,0,NaN,50,среднее,женат / замужем,F,госслужащий,0,NaN,сделка с подержанным автомобилем,операции с автомобилем,взрослый 35+
55,0,NaN,54,среднее,гражданский брак,F,пенсионер,1,NaN,сыграть свадьбу,проведение свадьбы,взрослый 35+


In [15]:
print(f'Доля пропущенных значений составляет {round(len(empty_str)/len(data) * 100, 1)}%')

Доля пропущенных значений составляет 10.1%


Можно предположить, что эти пропущенные значения не случайны, так как полностью совпадают друг с другом. 
- Возможно, эти данные были из одного источника типо лендинга или определенного кредитного продукта;
- Не исключена утеря данных или ошибка в их сборе;
- Нестабильный доход и неофициальная работа или ее отсутствие могли стать причиной для пропусков;

In [16]:
empty_str.pivot_table(
    index = 'income_type',
    columns = 'category_purpose',
    values = 'debt',
    aggfunc = 'count'
)


category_purpose,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы,ремонт\строительство
income_type,,,,,
госслужащий,27.0,55.0,35.0,14.0,16.0
компаньон,93.0,186.0,105.0,55.0,67.0
пенсионер,82.0,155.0,73.0,58.0,42.0
предприниматель,NaN,1.0,NaN,NaN,NaN
сотрудник,216.0,428.0,212.0,120.0,124.0


In [17]:
empty_str.pivot_table(
    index = 'income_type',
    columns = 'category_age',
    values = 'children',
    aggfunc = 'sum'
)

category_age,взрослый 35+,молодой 18+,старый 60+
income_type,,,
госслужащий,38.0,63.0,0.0
компаньон,186.0,125.0,0.0
пенсионер,37.0,2.0,29.0
предприниматель,0.0,NaN,NaN
сотрудник,387.0,325.0,5.0


-  у клиетов очевидно не один и тот же кредитный продукт типо кредитки, так как отличаются кредитные цели;
-  в income_type есть 'пенсионер' и 'госслужащий' их доход обычно стабилен и не равен 0;
-  вероятно, отсутствие данных произошло случайно, тк в эти данные составляют 10% от остальной таблицы, то не будем их удалять, а заполним медиаными значениями, тк выборка касательно доходов и дней стажа сильно отличается по таблице.

In [18]:
#days_employed преобразуем значения, чтобы с ними можно было работать
data.days_employed = abs(data.days_employed)
data.days_employed.head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

У нас все еще есть значения NaN и поэтому в целочисленное значения days_employed будем осуществлять перевод, после того как заполним пропуски.

In [19]:
data.total_income = data.total_income.fillna(data.total_income.median()).astype('int')

#Проверим остались ли значения >=0
data[data.total_income <= 0].total_income.count()

0

Теперь, когда в столбце total_income нет аномалий и пропущенных значений отфильтруем его по категориям.

In [20]:
def category_income(total_income):
    if total_income < 30000:
        return 'E'
    elif total_income < 50000:
        return 'D'
    elif total_income < 200000:
        return 'C'
    elif total_income < 1000000:
        return 'B'
    else:
        return 'A'

data['category_income'] = data.total_income.apply(category_income)
data.groupby('category_income').total_income.count()

category_income
A       25
B     5015
C    16014
D      348
E       22
Name: total_income, dtype: int64

Обработаем столбец days_employed - стаж работы в днях. В отличие от дохода, стаж не может стремится бесконечности. days_employed ограничен сроком начала и окончания трудовой деятельности и она не может превышать возраст клиента. По информации admmegion.ru cтаж засчитывается в отдельных случаях с 14 лет, если был заключен официальный трудовой договор в том числе на территории бывшего СССР.

In [23]:
count_ureal_days = data[data.days_employed/365 > data.dob_years - 14]
print(f'Количество строк с нереалистичным стажем составляет {round(count_ureal_days.category_age.count()/len(data) * 100, 1)}% от всей таблицы')

Количество строк с нереалистичным стажем составляет 16.1% от всей таблицы


Есть несколько вариантов почему это могло произойти:
- Данные о стаже чаще заполняются в формате год\месяц и ошибка произошла в процессе форматирования;
- Данные были изначально некорректными;

In [24]:
print(f'Максимальное значение стажа в годах превышающих dob_years в годах: {count_ureal_days.days_employed.max()/365}')  
print(f'Минимальное значение стажа в годах превышающих dob_years в годах: {count_ureal_days.days_employed.min()/365}')  


Максимальное значение стажа в годах превышающих dob_years в годах: 1100.6997273296713
Минимальное значение стажа в годах превышающих dob_years в годах: 15.544841681382076


К сожалению, изменение столбца days_employed может еще сильнее его исказить, поэтому оставим их в этом виде, но заменим тип на целочисленный и заменим пропуски медиальным значением. В дальнейшем не будем опираться на эти данные, так как их корректность под вопросом.

In [25]:
data.days_employed.mean()

66902.43787703343

In [26]:
import numpy as np

# Замена кривых значений пропусками
data.loc[data.days_employed/365 > data.dob_years - 14, 'days_employed'] = np.nan

# Заполнение пропусков средним значением стажа по возрастной группе
data.days_employed = data.days_employed.fillna(data.groupby('category_age')['days_employed'].transform('mean'))

In [27]:
data.days_employed.mean()

2515.636355075652

In [28]:
data.days_employed = data.days_employed.astype('int')
data.days_employed.info()

<class 'pandas.core.series.Series'>
Index: 21424 entries, 0 to 21524
Series name: days_employed
Non-Null Count  Dtype
--------------  -----
21424 non-null  int32
dtypes: int32(1)
memory usage: 251.1 KB


То есть среднее по столбцу у тебя в итоге упало бы с 173 лет до 6.8 лет

-----

Проверим последний раз наличие пропусков:

In [29]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
family_status       0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
category_purpose    0
category_age        0
category_income     0
dtype: int64

In [30]:
data.duplicated().sum()

71

[описание проекта](#description)

### Поиск дубликатов<a id="duplicated"></a>

- проверим наличие скрытых дубликатов
- проверим дубликаты методом duplicated().sum()
- удалим дубликаты методом .drop_duplicates().reset_index(drop=True)

In [31]:
data.children.unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [32]:
data.children = abs(data.children)

In [33]:
data.children.unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

In [34]:
data.duplicated().sum()

71

In [35]:
data = data.drop_duplicates().reset_index(drop=True)

In [36]:
len(data)

21353

In [37]:
data = data.drop(['days_employed','dob_years','total_income'], axis=1)
data.head()

,children,education,family_status,gender,income_type,debt,purpose,category_purpose,category_age,category_income
0,1,высшее,женат / замужем,F,сотрудник,0,покупка жилья,операции с недвижимостью,взрослый 35+,B
1,1,среднее,женат / замужем,F,сотрудник,0,приобретение автомобиля,операции с автомобилем,взрослый 35+,C
2,0,среднее,женат / замужем,M,сотрудник,0,покупка жилья,операции с недвижимостью,молодой 18+,C
3,3,среднее,женат / замужем,M,сотрудник,0,дополнительное образование,получение образования,молодой 18+,B
4,0,среднее,гражданский брак,F,пенсионер,0,сыграть свадьбу,проведение свадьбы,взрослый 35+,C


Для подсчета дубликатов использовался методы duplicated().sum(), потому что в моем арсенале имеется этот метод и ручной посчет, так как последний при таком объеме данных не гарантирует быстрое и точное решения, поэтому был использован именно duplicated().sum()

[описание проекта](#description)

### Ответить на вопросы:<a id="answer"></a>
 Чтобы ответить на вопросы:
 - Проверим количество строк в каждой группе, чтобы понять какая разница в количестве данных и на какие из них не стоит опираться;
 - Проверим какой процент данных выделяется в таблицах с помощью среднего кол-ва значений debt по столбцам;
 - Попробуем подумать и интерпретировать как-нибудь

In [38]:
data.debt.value_counts()
print(f'Количество клиентов с фактом наличия задолженности по возврату составляет {round(data[data.debt >0].debt.count()/len(data) * 100)}% от предоставленных данных')

Количество клиентов с фактом наличия задолженности по возврату составляет 8% от предоставленных данных


In [39]:
data.category_income.value_counts()

category_income
C    15943
B     5015
D      348
A       25
E       22
Name: count, dtype: int64

Уберем данные, где меньше 100 человек в группе тк этих данных слишком мало для того чтобы делать выводы

In [40]:
data.children.value_counts()

children
0     14022
1      4839
2      2039
3       328
20       75
4        41
5         9
Name: count, dtype: int64

In [41]:
data = data[(data.groupby('children')['debt'].transform('count') > 100) & 
            (data.groupby('category_income')['debt'].transform('count') > 100) &
            (data.groupby('income_type')['debt'].transform('count') > 100)]


Возьмем столбец category_income и посмотрим как он влияет на возврат кредита в срок

In [42]:
round(data.pivot_table(
    index = 'children',
    columns = 'category_income',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_income,B,C,D
children,,,
0,6.8,7.8,5.0
1,7.5,9.7,6.0
2,8.2,9.8,11.8
3,4.5,9.4,20.0


In [43]:
data[(data.children == 3) & (data.category_income == 'B')].debt.value_counts()

debt
0    84
1     4
Name: count, dtype: int64

 - в категории С прослеживается, чем больше детей, тем больше % debt;
 - в категории D напротив,чем больше детей, тем больше % debt;
 - в категории B чем больше детей, тем больше % debt, исключение клиенты с 3 детьми там %debt меньше 5%;

In [44]:
round(data.pivot_table(
    index = 'category_age',
    columns = 'category_income',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_income,B,C,D
category_age,,,
взрослый 35+,6.7,7.7,5.8
молодой 18+,8.6,11.3,7.6
старый 60+,3.8,5.1,5.5


- возрастная категория 60+ так же имеет самый низкий % debt
- возрастная категория 18+ так же имеет самый высокий % debt в каждой категории дохода

In [45]:
round(data.pivot_table(
    index = 'category_purpose',
    columns = 'category_income',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_income,B,C,D
category_purpose,,,
операции с автомобилем,8.3,9.5,14.1
операции с недвижимостью,6.6,7.5,3.7
получение образования,7.8,9.9,2.6
проведение свадьбы,5.2,8.7,8.8
ремонт\строительство,7.3,7.2,3.2


- У категории D в большинстве category_purpose наименьший % debt и наибольший операции с авто и проведение свадьбы, наименьший образование;
- С образование max %debt -10% по всем категориям в этой таблице, в среднем по таблице наибольший %debt в этой категории min-7.2% ремонт;

In [46]:
round(data.pivot_table(
    index = 'family_status',
    columns = 'category_income',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_income,B,C,D
family_status,,,
в разводе,6.5,7.2,11.8
вдовец / вдова,5.8,6.9,0.0
гражданский брак,6.6,10.2,8.1
женат / замужем,6.8,7.8,5.4
не женат / не замужем,9.4,9.9,8.7


- категория С стабильно высокий debt % в сравнении с B и D,, наибольший в гр.брак наименьший вдовы;
- B высокий % не в браке низкий у вдовцов
- в категории D самый высокий % debt в разводе, самый низкий у вдов высокий не в браке и гр браке;

In [47]:
round(data.pivot_table(
    index = 'family_status',
    columns = 'category_purpose',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_purpose,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы,ремонт\строительство
family_status,,,,,
в разводе,7.6,6.4,7.2,NaN,8.7
вдовец / вдова,8.8,5.6,7.6,NaN,3.9
гражданский брак,11.9,9.3,14.8,7.9,9.5
женат / замужем,8.3,6.9,8.4,NaN,6.9
не женат / не замужем,13.0,8.2,10.9,NaN,7.5


- высокий dept% у клиентов по в гр.браке получение образования 14.8% низкий недвижимость 9.3%;
- не в браке (13.1% авто) 7.5% ремонт
- клиенты в разводе(8.7% ремонт & 6.4% недвиж) и вдовцы имеют низкий dept (самый высокий авто 8.8%, самый низкий ремонт 3.9%)
- брак (8.4% образование, 6.9% ремонт)

Проверим связь сп, наличия детей и debt%

In [48]:
round(data.pivot_table(
    index = 'children',
    columns = 'family_status',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
children,,,,,
0,7.1,6.2,8.3,6.9,9.4
1,6.7,8.6,11.9,8.1,11.3
2,8.9,15.0,8.8,9.5,12.2
3,9.1,0.0,14.3,6.9,12.5


Проверим столбец "вдовец/вдова" на количество строк

In [48]:
data[(data.family_status == 'вдовец / вдова') & (data.children == 3)].debt.value_counts()

debt
0    6
Name: count, dtype: int64

- гр брак как признак высокого %dept
- не женат/не замужем, в разводе(исключение 1 ребенок) - чем больше детей, тем больше риск увеличения %dept и возможно у вдов так же возможно
- в браке многодетные(3 ребенка) от 0 до 2 детей % растет, а при 3х % debt меньше, чем при 0

In [49]:
round(data.pivot_table(
    index = 'children',
    columns = 'category_purpose',
    values = 'debt',
    aggfunc = 'mean') * 100, 1)

category_purpose,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы,ремонт\строительство
children,,,,,
0,8.5,6.8,8.7,7.5,6.5
1,10.6,8.1,10.3,9.6,7.6
2,11.8,8.2,11.5,5.6,9.9
3,8.3,7.0,5.8,15.6,10.5


- Наименьший %debt в среднем по строке с бездетными до 8.7%(образование)
- 1 ребенок 7.6% ремонт - 10.6% авто
- 2 ребенок 11.8% авто 11.5% образование - 5.6% свадьба
- 3 ребенка 15.6% свадьба - 5.8% образование
- ремонт %debt растет по возрастанию детей, а авто, недвижимость, образование до 3 ребенка 

[описание проекта](#description)

#### Категория D: 
##### Итог: стабильные;
#####  фактор риска: 
- наличие детей, возраст 60+; 
- рискованные цели: автокредит;
- развод, не женат\не замужем и гражданский брак.

#### Категория С: 
##### Итог: нестабильные;
##### факторы риска:
- наличие детей, возраст 18-35;
- цели: образование, автокредит;
- женат\замужем, гражданский брак
##### минимальный риск при:
- отсутствие детей
- возраст 60+
- строительство\недвижимость
- сп в разводе
#### Категория B:
 ##### Итог: средняя стабильные;
 ##### факторы риска:
- количество детей от 0 до 2;
- автокредиты;
- сп: не женат\не замужем
##### минимальный риск при:
- количество детей 3;
- возраст 60+;
- кредит на свадьбу;
- вдовцы

#### Дополнительные факторы:
     - Наибольший dept операции с автомобилем и получение образования
     - высокий %debt не женат\незамужем и в гражданском браке, низкий у вдовцов и в разводе;
     -  сп не в браке, в разводе, вдове\вдова - чем больше детей, тем больше риск увеличения %dept
     - при 3х детях высокий %debt на проведение свадьбы

In [50]:
round(data.groupby(['category_purpose', 'family_status']).debt.mean() * 100, 1)

category_purpose          family_status        
операции с автомобилем    в разводе                 7.6
                          вдовец / вдова            8.8
                          гражданский брак         11.9
                          женат / замужем           8.3
                          не женат / не замужем    13.0
операции с недвижимостью  в разводе                 6.4
                          вдовец / вдова            5.6
                          гражданский брак          9.3
                          женат / замужем           6.9
                          не женат / не замужем     8.2
получение образования     в разводе                 7.2
                          вдовец / вдова            7.6
                          гражданский брак         14.8
                          женат / замужем           8.4
                          не женат / не замужем    10.9
проведение свадьбы        гражданский брак          7.9
ремонт\строительство      в разводе                 8.7


[описание проекта](#description)

### Вывод:<a id="result"></a>

 ##### В скоринговой системе важно учесть семейное положение, количество детей, а так же уровень дохода, учитывая следующие факторы:
- При наличии детей процент невозврата кредита в срок больше, чем при их отсутствии. Исключением является категория B(доход 200тыс–1млн) при 3х детях эти клиенты имеют самый низкий % по невозврату кредита в срок; Так же одинокие клиенты(сп: неженат\незамужем, в разводе): При большем количестве детей имеют больший процент невозврата кредита в срок.
##### больший процент невозврата кредита в срок у клиентов: 
- СП: не женат\незамужем и в гражданском браке;
- Доход: с доходом от 50тыс до 200тыс;
- Цели: операции с автомобилем и получение образования;
##### наименьший процент невозврата кредита в срок у клиентов: 
- СП: вдовы\вдовцы
- Доход: клиенты с доходом до 50тыс
- Цели: недвижимость, ремонт.